In [10]:
from dotenv import load_dotenv
load_dotenv()
import rich

In [1]:
from langfuse import Langfuse
langfuse = Langfuse()

In [3]:
langfuse.create_prompt(
    name="event-planner-chat",
    prompt=[
        {"role":"system", "content":"You are an senior event planner."},
        {"role":"user", "content": 
            "Plan an event titled {{Event_Name}}. The event will be about: {{Event_Description}}. "
            "The event will be held in {{Location}} on {{Date}}. "
            "Consider the following factors: audience, budget, venue, catering options, and entertainment. "
            "Provide a detailed plan including potential vendors and logistics.",
        }
    ],
    type="chat",
    config={
        "model":"gpt-4o-mini",
        "temperature": 0,
    },
    labels=["production"]
)

In [31]:
langfuse_chat_prompt_template = langfuse.get_prompt(
    name="event-planner-chat",
    type="chat",
)

In [32]:
rich.print(langfuse_chat_prompt_template.prompt)
rich.print(langfuse_chat_prompt_template.config)

[
    {'role': 'system', 'content': 'You are an senior event planner.'},
    {
        'role': 'user',
        'content': 'Plan an event titled {{Event_Name}}. The event will be about: {{Event_Description}}. The event 
will be held in {{Location}} on {{Date}}. Consider the following factors: audience, budget, venue, catering 
options, and entertainment. Provide a detailed plan including potential vendors and logistics.'
    }
]

{'model': 'gpt-4o-mini', 'temperature': 0}

In [14]:
langfuse_chat_prompt_template.compile(Event_Name="My Birthday Party", Event_Description="A fun party", Location="My house", Date="Saturday")

[{'content': 'You are an senior event planner.', 'role': 'system'},
 {'content': 'Plan an event titled My Birthday Party. The event will be about: A fun party. The event will be held in My house on Saturday. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.',
  'role': 'user'}]

In [20]:
langchain_prompt_template = langfuse_chat_prompt_template.get_langchain_prompt()
print(langchain_prompt_template)
print(type(langchain_prompt_template))

[('system', 'You are an senior event planner.'), ('user', 'Plan an event titled {Event_Name}. The event will be about: {Event_Description}. The event will be held in {Location} on {Date}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.')]
<class 'list'>


In [15]:
from langchain_core.prompts import ChatPromptTemplate

In [21]:
langchain_prompt_template = ChatPromptTemplate.from_messages(
    messages=langfuse_chat_prompt_template.get_langchain_prompt(),
    )

In [33]:
langchain_prompt_template.metadata = {"langfuse_prompt": langfuse_chat_prompt_template}

In [34]:
langchain_prompt_template

ChatPromptTemplate(input_variables=['Date', 'Event_Description', 'Event_Name', 'Location'], input_types={}, partial_variables={}, metadata={'langfuse_prompt': <langfuse.model.ChatPromptClient object at 0x1050a82f0>}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an senior event planner.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Date', 'Event_Description', 'Event_Name', 'Location'], input_types={}, partial_variables={}, template='Plan an event titled {Event_Name}. The event will be about: {Event_Description}. The event will be held in {Location} on {Date}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.'), additional_kwargs={})])

In [24]:
model = langfuse_chat_prompt_template.config["model"]
temperature = str(langfuse_chat_prompt_template.config["temperature"])
print(f"Prompt model configurations\nModel: {model}\nTemperature: {temperature}")

Prompt model configurations
Model: gpt-4o-mini
Temperature: 0


In [35]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, temperature=temperature)

chain = langchain_prompt_template | llm

In [26]:
example_input = {
    "Event_Name": "Wedding",
    "Event_Description": "The wedding of Julia and Alex, a charming couple who share a love for art and nature. This special day will celebrate their journey together with a blend of traditional and contemporary elements, reflecting their unique personalities.",
    "Location": "Central Park, New York City",
    "Date": "June 5, 2024"
}

In [27]:
from langfuse.callback import CallbackHandler
langfuse_callback_handler = CallbackHandler()

In [37]:
# we pass the callback handler to the chain to trace the run in Langfuse
response = chain.invoke(input=example_input,config={"callbacks":[langfuse_callback_handler]})
# response = chain.invoke(input=example_input)

print(response.content)

### Event Plan: Wedding of Julia and Alex

**Event Title:** Wedding of Julia and Alex  
**Date:** June 5, 2024  
**Location:** Central Park, New York City  
**Theme:** A blend of traditional and contemporary elements celebrating art and nature.

---

### 1. Audience
- **Guests:** Approximately 150-200 close family and friends.
- **Demographics:** A mix of young adults, families, and older relatives, with a focus on art and nature enthusiasts.

### 2. Budget
- **Estimated Total Budget:** $50,000
  - Venue Permit: $2,000
  - Catering: $20,000
  - Decor: $10,000
  - Entertainment: $5,000
  - Photography/Videography: $5,000
  - Attire: $5,000
  - Miscellaneous (invitations, transportation, etc.): $3,000

### 3. Venue
- **Location:** Central Park, specifically the Conservatory Garden or the Great Lawn.
- **Permit:** Apply for a special event permit through the NYC Parks Department (approximately $2,000).
- **Logistics:** 
  - Set up a designated area for the ceremony and reception.
  - Cons

In [30]:
llm.invoke("Plan an event titled Wedding. The event will be about: The wedding of Julia and Alex, a charming couple who share a love for art and nature. This special day will celebrate their journey together with a blend of traditional and contemporary elements, reflecting their unique personalities.. The event will be held in Central Park, New York City on June 5, 2024. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.")

AIMessage(content="### Event Plan: Wedding of Julia and Alex\n\n**Event Title:** Wedding of Julia and Alex\n\n**Date:** June 5, 2024\n\n**Location:** Central Park, New York City\n\n---\n\n### 1. Audience\n- **Guests:** Approximately 150-200 close family and friends.\n- **Demographics:** A mix of young adults, families, and older relatives, reflecting Julia and Alex's diverse social circles.\n\n### 2. Budget\n- **Estimated Total Budget:** $50,000\n  - **Venue Permit:** $2,000\n  - **Catering:** $20,000\n  - **Decorations:** $8,000\n  - **Photography/Videography:** $5,000\n  - **Entertainment:** $5,000\n  - **Attire (Bride & Groom):** $5,000\n  - **Miscellaneous (invitations, transportation, etc.):** $5,000\n\n### 3. Venue\n- **Location:** Central Park, specifically the Conservatory Garden or the Great Lawn.\n- **Permit:** Apply for a wedding permit through the NYC Parks Department. Ensure to book at least 6 months in advance.\n- **Logistics:** \n  - Arrange for seating (chairs/tables) a